# Conexion

In [63]:
# Variables de entorno
import os
from dotenv import load_dotenv

# Cargar las variables desde el archivo .env
load_dotenv()

# Obtener las variables desde el entorno
account_name = os.getenv("VTEX_ACCOUNT_NAME")
app_key = os.getenv("VTEX_APP_KEY")
app_token = os.getenv("VTEX_APP_TOKEN")

# Validar que las variables se cargaron correctamente
if not all([account_name, app_key, app_token]):
    raise ValueError("Faltan credenciales en el archivo .env")

In [64]:
# import sys
# sys.path.append(r"C:\Users\Usuario\AppData\Local\Programs\Python\Python312\Lib\site-packages")
import requests # peticiones HTML
import pandas as pd # manejo de datos
import matplotlib.pyplot as plt # graficacion

In [65]:
# Configuración de la URL y headers
base_url = f"https://{account_name}.vtexcommercestable.com.br/api/oms/pvt/orders"
headers = {
    "X-VTEX-API-AppKey": app_key,
    "X-VTEX-API-AppToken": app_token,
    "Content-Type": "application/json",
}

In [71]:
# Parámetros para filtrar órdenes - Se puede obtener de VTEX, compartiendo filtros
params = {
    "date":
      {
        # "f_creationDate": "custom:[2023-12-31T03:00:00.000Z TO 2024-12-31T02:59:59.999Z]"
        "f_creationDate": "current-year"
      },
    "page": 1, # de 1 a 30
    "per_page": 1000, # de 1 a 100
}

response = requests.get(base_url, headers=headers, params=params)

response

<Response [200]>

In [67]:
# PARA IMPRIMIR TXT RESULTADO
# ordersTxt = response.json()

# Guardar la respuesta en un archivo .txt
# with open("respuesta_ordenes.txt", "w") as file:
#    file.write(str(ordersTxt))  # Convertir el diccionario a una cadena de texto

# Este código guardará la respuesta en un archivo llamado "respuesta_ordenes.txt"

In [68]:
import pandas as pd

# SE ARMA DATA FRAME
df = pd.DataFrame()

# Procesar los datos de la respuesta
if response.status_code == 200:
    orders = response.json()["list"]  # Extraemos la lista de órdenes

    # Verificar si hay órdenes
    if orders:
        # Crear un DataFrame a partir de las órdenes
        dfOrig = pd.DataFrame(orders)
        
        # Filtrar las columnas relevantes
        df = dfOrig[["sequence", "creationDate", "totalItems", "totalValue", "paymentNames", "status", "hostname", "origin"]]
        
        # Convertir la columna 'creationDate' a tipo datetime sin zona horaria
        df.loc[:, "creationDate"] = pd.to_datetime(df["creationDate"]).dt.strftime('%Y-%m-%d %H:%M:%S')  # Solo mostrar fecha y hora, sin la zona horaria
        
        # Convertir 'totalValue' de centavos a la moneda original (asumiendo que el valor está en centavos)
        df.loc[:, "totalValue"] = df["totalValue"] / 100
        
    else:
        print("No se encontraron órdenes.")
else:
    print(f"Error al obtener datos: {response.status_code}")
    print("Detalles del error:", response.text)
    
# Mostrar las primeras filas para verificar los datos
df.head()

,sequence,creationDate,totalItems,totalValue,paymentNames,status,hostname,origin
0,2216701,2024-12-30 16:44:25,1,22409.1,Visa Electron,window-to-cancel,newsport,Marketplace
1,2216699,2024-12-30 16:42:18,1,95499.0,GOcuotas,window-to-cancel,cero26,Marketplace
2,2216697,2024-12-30 16:39:32,1,124999.0,MercadoPagoPro,window-to-cancel,newsport,Marketplace
3,2216695,2024-12-30 16:38:33,2,211999.0,Visa,ready-for-handling,newsport,Marketplace
4,2216693,2024-12-30 16:37:38,2,363998.0,Naranja,ready-for-handling,newsport,Marketplace


In [72]:
df.to_csv("ventas.csv", index=False)